# Fasttext Embedding

In [1]:
from gensim.models import KeyedVectors
import json
import numpy as np
import torch
from sklearn.metrics import f1_score
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [2]:
label_map = {'B_GPE', 'B_PETITIONER', 'B_ORG', 'I_CASE_NUMBER', 'I_ORG', 'I_STATUTE', 'B_RESPONDENT', 'I_JUDGE', 'I_GPE', 'B_COURT', 'I_DATE', 'B_OTHER_PERSON', 'B_PRECEDENT', 'B_JUDGE', 'O', 'I_PETITIONER', 'I_OTHER_PERSON', 'B_STATUTE', 'I_RESPONDENT', 'B_WITNESS', 'B_CASE_NUMBER', 'I_COURT', 'B_DATE', 'I_WITNESS', 'I_PROVISION', 'I_PRECEDENT', 'B_PROVISION'}

In [3]:
word2vec_model_path = 'crawl-300d-2M.vec'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, limit = 100000)


with open('NER_train.json', 'r') as file:
    dataset = json.load(file)

print(len(dataset))

max_seq_len = 315
word_embeddings = []
labels = []

for key, value in dataset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text] 
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)
print(word_embeddings_array.shape)
texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
labels_tensor = torch.tensor(labels)

8019
(8019, 315, 300)


In [4]:
with open('NER_val.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 315

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

test_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
test_labels_tensor = torch.tensor(labels)

1416


In [5]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aman21010 (yuhuhu). Use `wandb login --relogin` to force relogin


True

In [6]:
wandb.init(
    project="nlp_ass2", 
    name=f"RNN_Fasttext"
)

In [7]:
class RNNTagger(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNTagger, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out

input_size = 300
hidden_size = 256
output_size = 100

model = RNNTagger(input_size, hidden_size, output_size)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 45
batch_size = 32 

texts_tensor = texts_tensor.to(device)
labels_tensor = labels_tensor.to(device)
val_texts_tensor = test_texts_tensor
val_labels_tensor =  test_labels_tensor
val_texts_tensor = val_texts_tensor.to(device)
val_labels_tensor = val_labels_tensor.to(device)

save_loss = 0

for epoch in range(num_epochs):
    
    model.train()  
    total_loss=0
    train_predictions = []
    train_labels = []
    for i in range(0, len(texts_tensor), batch_size):
        optimizer.zero_grad()
        batch_texts = texts_tensor[i:i+batch_size]
        batch_labels = labels_tensor[i:i+batch_size].view(-1)
        outputs = model(batch_texts)
        loss = criterion(outputs.view(-1, output_size), batch_labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        train_predictions.extend(torch.argmax(outputs, dim=2).flatten().cpu().tolist())
        train_labels.extend(batch_labels.cpu().tolist())
        
    train_f1 = f1_score(train_labels, train_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss/len(texts_tensor):.4f}, Training F1 Score: {train_f1:.4f}')
    
    # Validation
    model.eval()  
    total_val_loss = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for i in range(0, len(val_texts_tensor), batch_size):
            batch_texts_val = val_texts_tensor[i:i+batch_size]
            batch_labels_val = val_labels_tensor[i:i+batch_size].view(-1)
            outputs_val = model(batch_texts_val)
            val_loss = criterion(outputs_val.view(-1, output_size), batch_labels_val)
            total_val_loss += val_loss.item()
            val_predictions.extend(torch.argmax(outputs_val, dim=2).flatten().cpu().tolist())
            val_labels.extend(batch_labels_val.cpu().tolist())
    
    val_f1 = f1_score(val_labels, val_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss/len(val_texts_tensor):.4f}, Validation F1 Score: {val_f1:.4f}')
    if(val_f1>save_loss):
        save_loss = val_f1
        torch.save(model.state_dict(),'t1_rnn_fasttext.pt')
    log_metric = {"Epoch": epoch+1, "Training Loss": total_loss/len(texts_tensor), "Training F1 Score": train_f1, "Validation Loss": total_val_loss/len(val_texts_tensor), "Validation F1 Score": val_f1}
    wandb.log(log_metric)
print("Finished Training")
    

Epoch [1/45], Training Loss: 0.0099, Training F1 Score: 0.0216
Epoch [1/45], Validation Loss: 0.0033, Validation F1 Score: 0.0863
Epoch [2/45], Training Loss: 0.0028, Training F1 Score: 0.1911
Epoch [2/45], Validation Loss: 0.0022, Validation F1 Score: 0.2962
Epoch [3/45], Training Loss: 0.0019, Training F1 Score: 0.3468
Epoch [3/45], Validation Loss: 0.0017, Validation F1 Score: 0.3825
Epoch [4/45], Training Loss: 0.0017, Training F1 Score: 0.4155
Epoch [4/45], Validation Loss: 0.0015, Validation F1 Score: 0.4243
Epoch [5/45], Training Loss: 0.0014, Training F1 Score: 0.4511
Epoch [5/45], Validation Loss: 0.0014, Validation F1 Score: 0.4530
Epoch [6/45], Training Loss: 0.0013, Training F1 Score: 0.4878
Epoch [6/45], Validation Loss: 0.0013, Validation F1 Score: 0.4901
Epoch [7/45], Training Loss: 0.0013, Training F1 Score: 0.5150
Epoch [7/45], Validation Loss: 0.0013, Validation F1 Score: 0.5090
Epoch [8/45], Training Loss: 0.0012, Training F1 Score: 0.5371
Epoch [8/45], Validation Lo

In [8]:
with open('NER_test.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 315

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

test_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
test_labels_tensor = torch.tensor(labels)

with torch.no_grad():
    model = RNNTagger(input_size, hidden_size, output_size)
    model = model.to(device)
    model.load_state_dict(torch.load('t1_rnn_fasttext.pt'))
    test_outputs = model(test_texts_tensor.to(device)) 
    argmax_indices = torch.argmax(test_outputs, dim=2)
    reshaped_tensor = argmax_indices.view(949, 315)
    fl_out = reshaped_tensor.flatten()
    fl_label = test_labels_tensor.flatten().to(device) 
    f1 = f1_score(fl_out.cpu(), fl_label.cpu(), average='macro') 
    print("F1 Score on Test Set:", f1)

949
F1 Score on Test Set: 0.6401471565173543


In [9]:
wandb.init(
    project="nlp_ass2", 
    name=f"LSTM_Fasttext"
)

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Training F1 Score,▁▂▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████
Training Loss,█▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation F1 Score,▁▄▅▅▆▆▆▇▇▇▇▇█▇███▅████████████▇█████████
Validation Loss,▇▄▃▂▂▂▂▁▁▂▁▁▁▂▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▂
Epoch,45
Training F1 Score,0.83241
Training Loss,0.00047
Validation F1 Score,0.6337
Validation Loss,0.00137


In [10]:
class LSTMTagger(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMTagger, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

input_size = 300
hidden_size = 256
output_size = 100

model = LSTMTagger(input_size, hidden_size, output_size) # Move model to GPU if available

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 45
batch_size = 32 

texts_tensor = texts_tensor.to(device)
labels_tensor = labels_tensor.to(device)
val_texts_tensor = test_texts_tensor
val_labels_tensor =  test_labels_tensor
val_texts_tensor = val_texts_tensor.to(device)
val_labels_tensor = val_labels_tensor.to(device)

save_loss = 0

for epoch in range(num_epochs):
    
    model.train()  
    total_loss=0
    train_predictions = []
    train_labels = []
    for i in range(0, len(texts_tensor), batch_size):
        optimizer.zero_grad()
        batch_texts = texts_tensor[i:i+batch_size]
        batch_labels = labels_tensor[i:i+batch_size].view(-1)
        outputs = model(batch_texts)
        loss = criterion(outputs.view(-1, output_size), batch_labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        train_predictions.extend(torch.argmax(outputs, dim=2).flatten().cpu().tolist())
        train_labels.extend(batch_labels.cpu().tolist())
        
    train_f1 = f1_score(train_labels, train_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss/len(texts_tensor):.4f}, Training F1 Score: {train_f1:.4f}')
    
    # Validation
    model.eval()  
    total_val_loss = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for i in range(0, len(val_texts_tensor), batch_size):
            batch_texts_val = val_texts_tensor[i:i+batch_size]
            batch_labels_val = val_labels_tensor[i:i+batch_size].view(-1)
            outputs_val = model(batch_texts_val)
            val_loss = criterion(outputs_val.view(-1, output_size), batch_labels_val)
            total_val_loss += val_loss.item()
            val_predictions.extend(torch.argmax(outputs_val, dim=2).flatten().cpu().tolist())
            val_labels.extend(batch_labels_val.cpu().tolist())
    
    val_f1 = f1_score(val_labels, val_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss/len(val_texts_tensor):.4f}, Validation F1 Score: {val_f1:.4f}')
    if(val_f1>save_loss):
        save_loss = val_f1
        torch.save(model.state_dict(),'t1_lstm_fasttext.pt')
    log_metric = {"Epoch": epoch+1, "Training Loss": total_loss/len(texts_tensor), "Training F1 Score": train_f1, "Validation Loss": total_val_loss/len(val_texts_tensor), "Validation F1 Score": val_f1}
    wandb.log(log_metric)
print("Finished Training")
    

Epoch [1/45], Training Loss: 0.0103, Training F1 Score: 0.0620
Epoch [1/45], Validation Loss: 0.0026, Validation F1 Score: 0.2296
Epoch [2/45], Training Loss: 0.0019, Training F1 Score: 0.3370
Epoch [2/45], Validation Loss: 0.0018, Validation F1 Score: 0.3909
Epoch [3/45], Training Loss: 0.0015, Training F1 Score: 0.4187
Epoch [3/45], Validation Loss: 0.0016, Validation F1 Score: 0.4441
Epoch [4/45], Training Loss: 0.0013, Training F1 Score: 0.4636
Epoch [4/45], Validation Loss: 0.0014, Validation F1 Score: 0.4893
Epoch [5/45], Training Loss: 0.0012, Training F1 Score: 0.5089
Epoch [5/45], Validation Loss: 0.0013, Validation F1 Score: 0.5315
Epoch [6/45], Training Loss: 0.0011, Training F1 Score: 0.5491
Epoch [6/45], Validation Loss: 0.0013, Validation F1 Score: 0.5743
Epoch [7/45], Training Loss: 0.0010, Training F1 Score: 0.5898
Epoch [7/45], Validation Loss: 0.0012, Validation F1 Score: 0.5855
Epoch [8/45], Training Loss: 0.0010, Training F1 Score: 0.6196
Epoch [8/45], Validation Lo

In [11]:
with open('NER_test.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 315

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

test_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
test_labels_tensor = torch.tensor(labels)

with torch.no_grad():
    model = LSTMTagger(input_size, hidden_size, output_size)
    model = model.to(device)
    model.load_state_dict(torch.load('t1_lstm_fasttext.pt'))
    test_outputs = model(test_texts_tensor.to(device)) 
    argmax_indices = torch.argmax(test_outputs, dim=2)
    reshaped_tensor = argmax_indices.view(949, 315)
    fl_out = reshaped_tensor.flatten()
    fl_label = test_labels_tensor.flatten().to(device) 
    f1 = f1_score(fl_out.cpu(), fl_label.cpu(), average='macro') 
    print("F1 Score on Test Set:", f1)

949
F1 Score on Test Set: 0.6768922392802893


In [12]:
wandb.init(
    project="nlp_ass2", 
    name=f"GRU_Fasttext"
)

Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Training F1 Score,▁▃▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████
Training Loss,█▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation F1 Score,▁▄▄▅▆▆▇▇▇▇▇███████████████████▇▇▇▇▇▇▇▇▇▇
Validation Loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▅▅▆▆▅▅▆▆
Epoch,45
Training F1 Score,0.95903
Training Loss,0.00012
Validation F1 Score,0.62738
Validation Loss,0.00229


In [13]:
class GRUTagger(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUTagger, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.gru1 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.5) 

    def forward(self, x):
        out, _ = self.gru(x)
        out = torch.relu(out)
        out, _ = self.gru1(out)
        out = torch.relu(out)
        out = self.dropout(out)  
        out = self.fc1(out)
        return out
    
input_size = 300
hidden_size = 256
output_size = 27

model = GRUTagger(input_size, hidden_size, output_size)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 45
batch_size = 32 

texts_tensor = texts_tensor.to(device)
labels_tensor = labels_tensor.to(device)
val_texts_tensor = test_texts_tensor
val_labels_tensor =  test_labels_tensor
val_texts_tensor = val_texts_tensor.to(device)
val_labels_tensor = val_labels_tensor.to(device)

save_loss = 0

for epoch in range(num_epochs):
    
    model.train()  
    total_loss=0
    train_predictions = []
    train_labels = []
    for i in range(0, len(texts_tensor), batch_size):
        optimizer.zero_grad()
        batch_texts = texts_tensor[i:i+batch_size]
        batch_labels = labels_tensor[i:i+batch_size].view(-1)
        outputs = model(batch_texts)
        loss = criterion(outputs.view(-1, output_size), batch_labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        train_predictions.extend(torch.argmax(outputs, dim=2).flatten().cpu().tolist())
        train_labels.extend(batch_labels.cpu().tolist())
        
    train_f1 = f1_score(train_labels, train_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss/len(texts_tensor):.4f}, Training F1 Score: {train_f1:.4f}')
    
    # Validation
    model.eval()  
    total_val_loss = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for i in range(0, len(val_texts_tensor), batch_size):
            batch_texts_val = val_texts_tensor[i:i+batch_size]
            batch_labels_val = val_labels_tensor[i:i+batch_size].view(-1)
            outputs_val = model(batch_texts_val)
            val_loss = criterion(outputs_val.view(-1, output_size), batch_labels_val)
            total_val_loss += val_loss.item()
            val_predictions.extend(torch.argmax(outputs_val, dim=2).flatten().cpu().tolist())
            val_labels.extend(batch_labels_val.cpu().tolist())
    
    val_f1 = f1_score(val_labels, val_predictions, average='macro')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss/len(val_texts_tensor):.4f}, Validation F1 Score: {val_f1:.4f}')
    if(val_f1>save_loss):
        save_loss = val_f1
        torch.save(model.state_dict(),'t1_gru_fasttext.pt')
    log_metric = {"Epoch": epoch+1, "Training Loss": total_loss/len(texts_tensor), "Training F1 Score": train_f1, "Validation Loss": total_val_loss/len(val_texts_tensor), "Validation F1 Score": val_f1}
    wandb.log(log_metric)
print("Finished Training")
    

Epoch [1/45], Training Loss: 0.0087, Training F1 Score: 0.0675
Epoch [1/45], Validation Loss: 0.0037, Validation F1 Score: 0.0697
Epoch [2/45], Training Loss: 0.0029, Training F1 Score: 0.0982
Epoch [2/45], Validation Loss: 0.0026, Validation F1 Score: 0.1166
Epoch [3/45], Training Loss: 0.0022, Training F1 Score: 0.2177
Epoch [3/45], Validation Loss: 0.0020, Validation F1 Score: 0.3548
Epoch [4/45], Training Loss: 0.0017, Training F1 Score: 0.3689
Epoch [4/45], Validation Loss: 0.0015, Validation F1 Score: 0.4374
Epoch [5/45], Training Loss: 0.0014, Training F1 Score: 0.4347
Epoch [5/45], Validation Loss: 0.0014, Validation F1 Score: 0.4678
Epoch [6/45], Training Loss: 0.0012, Training F1 Score: 0.4688
Epoch [6/45], Validation Loss: 0.0013, Validation F1 Score: 0.4859
Epoch [7/45], Training Loss: 0.0011, Training F1 Score: 0.4999
Epoch [7/45], Validation Loss: 0.0012, Validation F1 Score: 0.5016
Epoch [8/45], Training Loss: 0.0010, Training F1 Score: 0.5371
Epoch [8/45], Validation Lo

In [14]:
with open('NER_test.json', 'r') as file:
    valset = json.load(file)

print(len(valset))
max_seq_len = 315

word_embeddings = []
labels = []

for key, value in valset.items():
    text = value['text']
    label_seq = value['labels']

    label_list = list(label_map)  # Store index of each 27 classes created
    label_indices = [label_list.index(label) for label in label_seq]
    label_indices_padded = label_indices[:max_seq_len] + [0] * (max_seq_len - len(label_indices))
    labels.append(label_indices_padded)

    vectors = [word2vec_model[word] if word in word2vec_model else np.zeros(300) for word in text]
    vectors_padded = [np.pad(vec, (0, 300), constant_values=0)[:300] for vec in vectors]
    vectors_padded += [np.zeros(300)] * (max_seq_len - len(vectors_padded))
    word_embeddings.append(vectors_padded)

word_embeddings_array = np.array(word_embeddings)

test_texts_tensor = torch.tensor(word_embeddings_array, dtype=torch.float32)
test_labels_tensor = torch.tensor(labels)

with torch.no_grad():
    model = GRUTagger(input_size, hidden_size, output_size)
    model = model.to(device)
    model.load_state_dict(torch.load('t1_gru_fasttext.pt'))
    test_outputs = model(test_texts_tensor.to(device)) 
    argmax_indices = torch.argmax(test_outputs, dim=2)
    reshaped_tensor = argmax_indices.view(949, 315)
    fl_out = reshaped_tensor.flatten()
    fl_label = test_labels_tensor.flatten().to(device) 
    f1 = f1_score(fl_out.cpu(), fl_label.cpu(), average='macro') 
    print("F1 Score on Test Set:", f1)

949
F1 Score on Test Set: 0.6477282497034494
